<h1><center>Pattern Recognition using problem Solving CA1</center><h1>

In [1]:
# Voting Ensemble for Classification

from sklearn.model_selection import KFold
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
import pickle
import pandas as pd

In [2]:
xls    = pd.ExcelFile('X_val_scaled_SL.xlsx')
X_val = pd.read_excel(xls, 'X_val_scaled')
y_val = pd.read_excel(xls, 'y_val')

X_val  = X_val.drop(['waterpoint_type_9:Unprotected dug well','waterpoint_type_OTHER','extraction_type_3:Surface pump',
           'extraction_type_2:Submersible pump','extraction_type_OTHER','waterpoint_type_3:Tube well or borehole',
           'waterpoint_type_2:Protected dug well','extraction_type_4:Hydram pump','extraction_type_5:Gravity',
           'extraction_type_1:Hand pump','waterpoint_type_5:Sand/Sub-surface dam (with well or standpipe)'],axis=1)

dataset_test = X_val
dataset_test['water point functionality'] = y_val['water point functionality']

X_val = X_val.drop(['water point functionality'],axis=1)

In [17]:
seed = 18
kfold = KFold(n_splits=8, random_state=seed)

# create the sub models
estimators = []

model1 = pickle.load( open( "Random_Forest_trained_model_20F_SL.p", "rb" ) )
estimators.append(('RandomFoest', model1))

model2 = pickle.load( open( "Gradient_Boosting_Classifier_trained_model_20F_SL.p", "rb" ) )
estimators.append(('GradientBoost', model2))

model3 = pickle.load( open( "KNN_trained_model_20F_SL.p", "rb" ) )
estimators.append(('knn', model3))

#model4 = pickle.load( open( "SVM_trained_model_20F_SL.p", "rb" ) )
#estimators.append(('svm', model4))


In [18]:
# create the ensemble model
ensemble = VotingClassifier(estimators)

results = cross_val_score(ensemble, X_val, y_val, cv=kfold)
print(results.mean())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

0.744019425884682


In [6]:
# We can save the trained model clf using pickle 
s = pickle.dumps(ensemble)
pickle.dump( ensemble, open( "Ensemble_model_20F_SL.p", "wb" ) )